In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
import time
import pandas as pd
import numpy as np
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
os.environ['PATH']+= r'C:/SeleniumDrivers'

In [38]:
driver=webdriver.Chrome()
driver.get('https://world.openfoodfacts.org/country/india')

In [4]:
elements=driver.find_elements(By.CSS_SELECTOR,'div.list_product_name')

In [5]:
for i in elements:
    print(i.text)

Red bull energy drink - 250 ml
Sprite - coca cola - 750 ml
Heinz Tomato ketchup - 400ml
Thumbs Up - Coca Cola Company - 750ml
Sting - 250 ml
kinley mineral water - 1 l
Sprite - 2.25l
Mountain Dew - Pepsico
maaza - Hisdustan Unilever Limited - 600 ml
Kinley Mineral Water - 1 L
Limca - Coca-Cola
Thums Up (Coca-Cola)
HERSHEY’S CHOCOLATE SYRUP - 623 g
Cadbury gems - 7.9g
Taka tak - Haldiram's
Thums up - 250 ml
Chyawanprash - Dabur - 550 g
Good Day Butter Cookies - Britannia - 68g
Sugar cookie - CMB - 100 g
Frooti - 2 l
Diet coke - The Coca-Cola Company - 500 ml
Mountain dew - mountaindew - 2.25
7up - PepsiCo - 750 ml
Coca Cola - 750 ml
Rooh Afza - hamdard - 750 ml
Marie Gold - Britannia - 73 g
Pepsi - 1,5 L
Pepsi 2.25 l - Pepsico
Parle - 22g
Bourn vita - 500g
Eno, fruit salt lemon flavour - GSK - 5g
Maaza Original - - 1.2 l
coco cola company - sprite - 250 ml
Barritas de platano y chocolate - Alteza
Oreo
Frooti mango drink - parle Agro - 150ml
thumsup - recycle
ORSL Plus Electrolyte Drink 

In [39]:
product_link = driver.find_elements(By.CSS_SELECTOR, 'a.list_product_a')

In [40]:
product_linkies=[]
for i in product_link:
    product_linkies.append(i.get_attribute('href'))

In [41]:
def individual():
    try:
        prod_name = driver.find_element(By.CSS_SELECTOR, 'h2.title-1').text.strip()
        rows = driver.find_elements(By.XPATH, "//tbody/tr")
        if not prod_name or not rows:
            return pd.DataFrame()
        nutrition_data = {"Product Name": [prod_name]}
        for row in rows:
            nutrient = row.find_element(By.XPATH, "./td[1]").text.strip()
            as_sold_100g = row.find_element(By.XPATH, "./td[2]").text.strip()
            nutrition_data[nutrient] = [as_sold_100g]
        df = pd.DataFrame(nutrition_data)
        return df
    except NoSuchElementException:
        return pd.DataFrame()

In [44]:
master_df = pd.DataFrame()

# ... (previous code)
# ... (previous code)

# ... (previous code)

# Define the desired columns
desired_columns = ["Product Name", "Energy", "Fat", "Saturated fat", "Cholesterol", "Carbohydrates", "Sugars", "Fiber", "Proteins", "Salt"]

for product_link in product_linkies:
    main_window = driver.current_window_handle
    driver.execute_script("window.open();")
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(product_link)
    
    try:
        # Wait until the page is loaded, give it a timeout of 20 seconds
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//tbody/tr")))
        product_df = individual()
        
        # Keep only the desired columns if they exist
        product_df = product_df[desired_columns] if all(col.lower().replace(" ", "") in map(lambda x: x.lower().replace(" ", ""), product_df.columns) for col in desired_columns) else pd.DataFrame()
        
        if not product_df.empty:
            # Print the obtained DataFrame for debugging
            print(product_df)
            
            # Extract numeric values from columns with units (excluding columns with '%')
            unit_columns = ["Energy", "Fat", "Saturated fat", "Cholesterol", "Carbohydrates", "Sugars", "Fiber", "Proteins", "Salt"]
            for col in unit_columns:
                if "%" not in col:
                    product_df[col] = product_df[col].apply(lambda x: re.search(r'\d+(\.\d+)?', str(x)).group() if pd.notna(x) else "?")
            
            # Replace "< 0.1" with 0.1 in numeric columns
            product_df.replace('< 0.1', '0.1', inplace=True, regex=True)
            
            # Convert numeric values to numeric format
            product_df = product_df.apply(lambda x: pd.to_numeric(x, errors='coerce') if x.name != "Product Name" else x)
            
            # Extract both kJ and kcal from the Energy column
            product_df['Energy_kJ'] = product_df['Energy'].apply(lambda x: float(re.search(r'\d+(\.\d+)?', str(x)).group()) if pd.notna(x) else None)
            product_df['Energy_kcal'] = product_df['Energy'].apply(lambda x: float(re.search(r'\d+(\.\d+)?', str(x)).group()) * 0.239 if pd.notna(x) else None)
            
            # Print the obtained DataFrame for debugging
            print(product_df)
            
            master_df = pd.concat([master_df, product_df], ignore_index=True)
        else:
            print(f"No data for {product_link}")
            
    except TimeoutException:
        # Handle the case where the table is not found within the specified timeout
        print(f"Timeout: Table not found for {product_link}")
    
    driver.close()
    driver.switch_to.window(main_window)

# ... (remaining code)


                     Product Name             Energy  Fat Saturated fat  \
0  Red bull energy drink - 250 ml  194 kj\n(46 kcal)  0 g           0 g   

  Cholesterol Carbohydrates Sugars Fiber Proteins   Salt  
0        0 mg          11 g   11 g   0 g      0 g  0.1 g  
                     Product Name  Energy  Fat  Saturated fat  Cholesterol  \
0  Red bull energy drink - 250 ml     194    0              0            0   

   Carbohydrates  Sugars  Fiber  Proteins  Salt  Energy_kJ  Energy_kcal  
0             11      11      0         0   0.1      194.0       46.366  
No data for https://world.openfoodfacts.org/product/8901764032271/sprite-coca-cola
No data for https://world.openfoodfacts.org/product/87157246/heinz-tomato-ketchup
No data for https://world.openfoodfacts.org/product/8901764042270/thumbs-up-coca-cola-company
No data for https://world.openfoodfacts.org/product/8902080000227/sting
No data for https://world.openfoodfacts.org/product/19017648/kinley-mineral-water
No data for h

AttributeError: 'NoneType' object has no attribute 'group'

In [43]:
master_df

,Product Name,Energy,Fat,Saturated fat,Cholesterol,Carbohydrates,Sugars,Fiber,Proteins,Salt
0,Red bull energy drink - 250 ml,194,0.0,0.0,0,11.0,11,0,0.0,0.100
1,Cadbury gems - 7.9g,1,17.5,17.3,3,75.1,70,0,3.6,0.185


In [125]:
master_df.to_excel('output.xlsx', index=False)